# [프로젝트] Multimodal RAG - Part 1

최근에는 텍스트 이외의 데이터에 대해서도 RAG를 수행하기 위한 작업들이 많이 연구되고 있습니다.   

일반적인 문서 로더를 이용해 RAG를 수행할 경우, 텍스트만 활용하게 되어 정보의 부분적 손실이 발생하는데요.    

이번 프로젝트에서는 2024년 8월 공개된 오픈 소스 라이브러리 Docling을 이용해,  

이미지/표 등이 포함된 PDF 문서를 재구성하고, 이를 통해 RAG를 수행해 보겠습니다.  

**GPU 클라우드 T4를 설정해 주세요!**

## 라이브러리 설치

docling 라이브러리를 설치합니다.
https://github.com/DS4SD/docling

In [ ]:
!pip install langchain==1.0.3
!pip install docling langchain_huggingface sentence_transformers jsonlines langchain-google-genai langchain-community beautifulsoup4 langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.1 MB/s eta 0:00:00
  Prepar

docling은 PDF 데이터를 마크다운으로 변환합니다.      
텍스트 이외에도, 표와 이미지를 추출할 수 있습니다.

In [ ]:
# 기본 코드: Image를 제외한 텍스트를 마크다운으로 변경
# T4 GPU 기준 3분 소요
from docling.document_converter import DocumentConverter

source = "https://storage.googleapis.com/deepmind-media/gemma/Gemma3Report.pdf"  # PDF path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())  # output: "### Technical Report[...]"

[INFO] 2025-11-02 06:08:52,330 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-02 06:08:52,348 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-02 06:08:52,349 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-02 06:08:52,415 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-02 06:08:52,419 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-11-02 06:08:52,420 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-11-02 06:08:52,455 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-02 06:08:52,486 [RapidOCR] download_file.py:60: File exists and is valid: /usr/loc

len(pages)=1, 0-0
len(valid_pages)=1
len(valid_page_images)=1


[INFO] 2025-11-02 06:09:11,358 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/resources/fonts/FZYTK.TTF
[INFO] 2025-11-02 06:09:13,155 [RapidOCR] download_file.py:82: Download size: 3.09MB
[INFO] 2025-11-02 06:09:13,314 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/FZYTK.TTF


len(pages)=3, 1-3
len(valid_pages)=3
len(valid_page_images)=3
len(pages)=4, 4-7
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 8-11
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 12-15
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 16-19
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 20-23
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=1, 24-24
len(valid_pages)=1
len(valid_page_images)=1
<!-- image -->

## Gemma 3 Technical Report

Gemma Team, Google DeepMind 1

We introduce Gemma 3, a multimodal addition to the Gemma family of lightweight open models, ranging in scale from 1 to 27 billion parameters. This version introduces vision understanding abilities, a wider coverage of languages and longer context - at least 128K tokens. We also change the architecture of the model to reduce the KV-cache memory that tends to explode with long context. This is achieved by increasing the ratio of local to global attention layers, and keeping the 

Docling은 다음의 작업을 지원합니다.
1. 각 페이지를 이미지로 추출하기
2. 페이지에 포함된 각 이미지를 추출하기
3. 전체를 HTML/MD 형식으로 재구성하기

In [ ]:
import logging
import time
import re
import requests
from pathlib import Path
from urllib.parse import urlparse
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

IMAGE_RESOLUTION_SCALE = 2.0
_log = logging.getLogger(__name__)

def download_pdf(url, save_dir="downloads"):
    """URL에서 PDF 파일을 다운로드하여 로컬 경로를 반환"""
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        filename = url.split("/")[-1]

        if not filename.endswith('pdf'):
            filename+='.pdf'
        file_path = save_dir / filename

        with open(file_path, "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)

        return str(file_path)
    else:
        raise Exception(f"Failed to download file: {url} (Status code: {response.status_code})")

def is_url(path):
    """주어진 문자열이 URL인지 확인"""
    return re.match(r'https?://', path) is not None

def parse(path, output_dir='docling_result'):
    logging.basicConfig(level=logging.INFO)

    if is_url(path):  # URL이면 다운로드
        _log.info(f"Downloading PDF from {path}...")
        path = download_pdf(path)

    input_doc_path = Path(path)
    output_dir = Path(output_dir)

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )

    start_time = time.time()
    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem

    # 페이지 이미지 저장
    for page_no, page in conv_res.document.pages.items():
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")

    # 이미지/테이블 저장
    table_counter = 0
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = output_dir / f"{doc_filename}-table-{table_counter}.png"
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = output_dir / f"{doc_filename}-picture-{picture_counter}.png"
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

    # 전체 마크다운 저장(이미지는 utf8 형태로)
    md_filename = output_dir / f"{doc_filename}-with-images.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.EMBEDDED)

    # 전체 마크다운 저장(이미지는 Reference 형태로)
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

    # 이미지 포함된 HTML 저장
    html_filename = output_dir / f"{doc_filename}-with-image-refs.html"
    conv_res.document.save_as_html(html_filename, image_mode=ImageRefMode.REFERENCED)

    end_time = time.time() - start_time
    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")

# 실행 예시
parse("https://storage.googleapis.com/deepmind-media/gemma/Gemma3Report.pdf")


[INFO] 2025-11-02 06:09:37,686 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-02 06:09:37,700 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-02 06:09:37,701 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-02 06:09:37,766 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-02 06:09:37,769 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-11-02 06:09:37,770 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-11-02 06:09:37,806 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-02 06:09:37,855 [RapidOCR] download_file.py:60: File exists and is valid: /usr/loc

len(pages)=1, 0-0
len(valid_pages)=1
len(valid_page_images)=1
len(pages)=1, 1-1
len(valid_pages)=1
len(valid_page_images)=1
len(pages)=4, 2-5
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 6-9
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 10-13
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 14-17
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 18-21
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=3, 22-24
len(valid_pages)=3
len(valid_page_images)=3
